In [1]:
%%capture
!pip install huggingsound
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q ipython-autotime
!pip install -q accelerate optimum
!pip install moviepy
!pip install langchain
!pip install chromadb
!pip install sentence-transformers
!pip install imutils
!pip install llama-cpp-python
!pip install yandexcloud

In [2]:
%%capture
%cd ~
!git clone --recursive https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make LLAMA_CUBLAS=1 -j libllama.so

# HACK: Use custom compiled libllama.so
%cp ~/llama.cpp/libllama.so /opt/conda/lib/python3.10/site-packages/llama_cpp/libllama.so

In [3]:
%%capture
%cd /kaggle/
!mkdir tmp
%cd tmp
!wget https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/resolve/main/model-q4_K.gguf

In [24]:
import re
import os
import torch
import librosa
import time
import cv2
import imutils
import shutil
import glob
import argparse

import numpy as np

from pprint import pprint
from collections import defaultdict
from pydub import AudioSegment
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    Speech2TextProcessor,
    Speech2TextForConditionalGeneration,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    AutoModelForSpeechSeq2Seq, 
    AutoProcessor, 
    pipeline
)
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.llms import YandexGPT
from chromadb.config import Settings
from llama_cpp import Llama

In [6]:
TRANSCRIBER_ID = "openai/whisper-large-v2"

EMBEDDER_ID = "ai-forever/sbert_large_nlu_ru"

PATH_TO_AUDIO = "/kaggle/input/yaraaa"     

# API_KEY = "AQVN0k6NUUf9UZkghayg6kGpyI8tNybqaj58cU60"  

SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
SYSTEM_TOKEN = 1587
USER_TOKEN = 2188
BOT_TOKEN = 12435
LINEBREAK_TOKEN = 13


DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
TORCH_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

print(DEVICE)

cuda:0


# Запуск процесса

In [7]:
# ygpt = YandexGPT(api_key=API_KEY)

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDER_ID)

# openai/whisper-large-v2  
processor = AutoProcessor.from_pretrained(TRANSCRIBER_ID)
model_t = AutoModelForSpeechSeq2Seq.from_pretrained(
    TRANSCRIBER_ID, torch_dtype=TORCH_DTYPE, low_cpu_mem_usage=True, use_safetensors=True
)
model_t.to(DEVICE)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model_t,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=TORCH_DTYPE,
    device=DEVICE,
)

# сайга
n_ctx = 3000 
top_k = 40
top_p = 0.5
temperature = 0.05
repeat_penalty = 1.1


ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}


def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def get_system_tokens(model):
    system_message = {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
    return get_message_tokens(model, **system_message)

def chat_saiga(message, model):
    system_tokens = get_system_tokens(model)
    tokens = system_tokens
    
    message_tokens = get_message_tokens(model=model, role="user", content=message)
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens
    generator = model.generate(
        tokens,
        top_k = top_k,
        top_p = top_p,
        temp = temperature,
        repeat_penalty = repeat_penalty,
        reset = True
    )
    
    result_list = []
    for token in generator:
        token_str = model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)
        if token == model.token_eos():
            break
        result_list.append(token_str)
    return ''.join(result_list)

In [ ]:
%%capture
try:
    del model_s
except:
    pass

model_path = '/kaggle/tmp/model-q4_K.gguf'
n_ctx = 3000

model_s = Llama(
    model_path = model_path,
    n_ctx = n_ctx,
    n_gpu_layers=-1
)

In [10]:
# очистка терминов
def clear_output(output):
    '''Стандартизируем термины'''
    output = (re.sub("""термин|[\{\}:\n\r'"]""", "", output)).split()
    if len(output)<3 and output:
        return ' '.join(output)
    return 'None'

# landchain 
def build_index_big(text, chunk_size, chunk_overlap):
    '''База текстовых батчей для конспекта'''

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = text_splitter.split_documents(
        [Document(page_content=text)]
    )
    
    fixed_documents = [doc for doc in documents if doc]
    db = Chroma.from_documents(
        fixed_documents,
        embeddings,
        client_settings=Settings(
            anonymized_telemetry=False
        ),
    )
    return db

def sliding_window(lst, window_size, step_size):
    '''Создание окна для базы на основе таймкодов'''
    windows = []
    if len(lst) >= window_size:
        right = len(lst) - window_size + 1
    else: 
        right = len(lst)
    for i in range(0, right, step_size):
        windows.append(lst[i:i + window_size])
    return windows

def build_index_time(full_text, chunk_size, chunk_overlap):
    '''База текстовых батчей на основе таймкодов транскрибатора'''

    documents = []
    for chunk in sliding_window(full_text['chunks'], chunk_size, chunk_overlap):
        
        meta_data = (chunk[0]['timestamp'][0], chunk[-1]['timestamp'][0])
        chunk_text = ' '.join([element['text'] for element in chunk])
        documents.append(Document(page_content=chunk_text, metadata={'start':meta_data[0], 'end':meta_data[1]}))
    
    fixed_documents = [doc for doc in documents if doc]
    db = Chroma.from_documents(
        fixed_documents,
        embeddings,
        client_settings=Settings(
            anonymized_telemetry=False
        ),
    )
    return db

def retrieve(text, db, k_documents):
    '''Поиск ближайших батчей текста'''
    context = ""
    if db:
        retriever = db.as_retriever(search_kwargs={"k": k_documents})
        docs = retriever.get_relevant_documents(text)
        retrieved_docs = "\n\n".join([doc.page_content for doc in docs])
    return retrieved_docs

def first_retrieve(text, db):
    '''Вывод ближайшего элемента дб'''
    context = ""
    if db:
        retriever = db.as_retriever(search_kwargs={"k": 1})
        doc = retriever.get_relevant_documents(text)[0]
    return doc

In [11]:
files = os.listdir(PATH_TO_AUDIO)
mp3_files = [file for file in files if file.endswith('.mp3')]

# цикл по аудио в папке
for audio_file in mp3_files:
    
    audio_file = f'{PATH_TO_AUDIO}/{audio_file}'
    with torch.no_grad():
        audio = librosa.load(audio_file, sr=16_000)[0]
        full_text = pipe(audio, generate_kwargs={"language": "russian"})
    
    db = build_index_time(full_text, 10, 5)
    
    # ищем список терминов
    terms_list = []
    for batch in db.get()['documents']:

        term_prompt = f"""
Найди ключевой термин, для которого дано опеределение в данном тексте.
Важно: для термина должно быть дано определение в тексте.
Если термин с определением есть, выводи {{термин}}
Если термина с определением нет, то выводи {{None}}.


Текст:
{batch}
        """
        with torch.no_grad():
            output = chat_saiga(term_prompt, model_s)
        output = clear_output(output)
        terms_list.append(output)

    terms_list = set([t.lower() for t in terms_list])
    terms_list.discard('none')

    terms_dict_list = []
    for t in terms_list:
        terms_dict = defaultdict(str)

        chain_prompt = f"""{t} - это"""
        term_text = retrieve(chain_prompt, db, 5)
        
        definition_prompt = f"""
Составь определение термину "{t}" на основе текста
Вывод должен быть в формате термин - определение
Дай пожалуйста определение термину {t} - 


Текст:
{term_text}
        """
        meta_time = first_retrieve(f'{chain_prompt} {term_text}', db).metadata

        terms_dict['term'] = t 

        with torch.no_grad():
            output = chat_saiga(definition_prompt, model_s)
        output = output[len(t + ' - '):]

        terms_dict['definition'] = output
        terms_dict['start'] = meta_time['start']
        terms_dict['end'] = meta_time['end']

        terms_dict_list.append(terms_dict)

    questions = [
        "какое название темы",
        "причина изучать этот курс для студента",
        "какую пользу принесёт данный курс для студента",
        "на что влияет знание этой темы",
        "каких ошибок студент избежит освоив эту тему",
        "почему без данной темы нельзя двигаться дальше в профессии",
        "план данной лекции"
    ]

    introduction = ""
    for q in questions:
        retrieve_text = retrieve(q, db, 4)
        q_prompt = f"""
    Используй текст лекции ответь на вопрос:
    {q}?

    Текст лекции:
    {retrieve_text}
        """
        with torch.no_grad():
            output = chat_saiga(q_prompt, model_s)
            if output.find('bot')!=-1:
                output = output[:output.find('bot')]

            introduction += output.replace('Выход:', '') +'\n\n'
        
    for document_id in db.get()['ids']:
        db._collection.delete(ids=document_id)
    db.persist()   
    
    # увеличиваем батчи в бд
    db = build_index_big(full_text['text'], 2000, 30)
    
    batch_summ = []
    for batch in db.get()['documents']:
        summ_prompt = f"""
    Напиши краткое изложение для введенного текста. Выводи в формате Краткое изложение: {{изложение}}


    Текст:
    {batch}
    """

        with torch.no_grad():
            output = chat_saiga(summ_prompt, model_s).replace("\n", "")
            if output.find(':')!=-1:
                output = output[output.find(':')+2:] + '\n'
        batch_summ.append(output)
    batch_summ = ' '.join(batch_summ)
    
    for document_id in db.get()['ids']:
        db._collection.delete(ids=document_id)
    db.persist()  
    torch.cuda.empty_cache()
    

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


# Глоссарий
В словаре есть термин, опредление, таймкоды где о нем говорится

In [32]:
pprint([dict(d) for d in terms_dict_list])

[{'definition': 'процесс проверки программного обеспечения на соответствие '
                'требованиям и ожиданиям пользователей.',
  'end': 145.6,
  'start': 114.04,
  'term': 'тестирование'},
 {'definition': 'простой для понимания язык программирования с '
                'интерпретируемым синтаксисом, который позволяет исполнять код '
                'строка за строкой. Он также популярный язык, что означает '
                'большое сообщество разработчиков и энтузиастов, а также '
                'множество готовых библиотек для решения реальных задач. '
                'Python также используется во многих областях, включая '
                'веб-разработку, машинное обучение, научное исследование и '
                'многое другое.',
  'end': 20.16,
  'start': 0.0,
  'term': 'python'}]


# Конспект  
В конспекте выдержана требуемая структура

In [33]:
print(introduction+'\n'+ 'Основной контент:'+'\n'+batch_summ)

    Название темы: "Преимущества языка Python и его применения в различных областях"

Причина изучать этот курс для студента - это возможность получить знания в области веб-разработки, машинного обучения и искусственного интеллекта, анализа данных, автоматизации и тестирования с помощью языка программирования Python. Это позволит студенту приобретать навыки, необходимые для работы в различных областях информационных технологий, а также создавать собственные проекты и онлайн-сервисы.

    Данный курс принесет студенту множество пользы:

1. Улучшение навыков программирования на языке Python, который является одним из самых популярных и простых в использовании языков программирования.
2. Ознакомление с интерпретатором Python, его интерактивным режимом и запуском отдельных файлов с кодом.
3. Изучение базовых понятий языка, таких как литерал переменная, а также знакомство с IDE PyCharm и начало работы с ней.
4. Ознакомление с машинным обучением и искусственным интеллектом, которые являются 